## Preprocessing

In [805]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

train = pd.read_csv('/content/gdrive/My Drive/Kaggle/titanic/train.csv')
test  = pd.read_csv('/content/gdrive/My Drive/Kaggle/titanic/test.csv')          

def preprocessing():
    global data

    data = pd.concat([train, test], axis=0, ignore_index=True)

    del data['Name'],data['Cabin'],data['Ticket'],data['PassengerId']

    data["Age"] = data["Age"].fillna(data.groupby("Sex")["Age"].transform("median"))

    data['Embarked'] = data['Embarked'].fillna('S')
    data = pd.get_dummies(data ,columns=['Embarked'],dtype=np.float64)

    data['Fare'] = data['Fare'].fillna(data.groupby("Pclass")["Fare"].transform("mean"))

    data['Sex'].replace({'male':0 ,'female':1} ,inplace=True)



def Outliers():                                                             
    flt = (data['Fare'] >= 400) & (data['Survived'].notnull())
    data.drop(data[flt].index, inplace=True)


preprocessing()

Outliers()

## Normalize & Split

In [813]:
X = MinMaxScaler().fit_transform(data)
X = pd.DataFrame(X)

train = X[X[0].notnull()]
Test = X[X[0].isnull()]

Test = Test.iloc[: ,1:]             # Predict It For Kaggel

X = train.iloc[:,1:]                # features
y = train.iloc[: ,0]                # Target

x_train , x_test , y_train , y_test = train_test_split(X, y, test_size=0.17 , random_state=42 )             
x_train.shape ,y_train.shape

((737, 9), (737,))

## Modeling

In [ ]:
tree = DecisionTreeClassifier(max_depth=4, random_state=42, ccp_alpha=0.001)

model = BaggingClassifier(base_estimator=tree, n_estimators=70, oob_score=True,   
                        max_samples=0.75,                                          
                        n_jobs=12 ,random_state=42)                 

model.fit(x_train, y_train)
model.score(x_test, y_test)

Predict = model.predict(TEST)
Kaggle = pd.DataFrame(DATA[DATA['Survived'].isnull()]['PassengerId'])
Kaggle ['Survived'] = Predict
Kaggle ['Survived'] = Kaggle ['Survived'].astype(int)
# Kaggle.to_csv('/kaggle/working/Bagging_.csv' ,index=False)

model.score(x_test, y_test)